# Assignment 03: Distribution generation

<html>
<div class="alert alert-info" role="alert" style="margin-top: 10px">
The goal of this exercise is to create some special distributions and plot the cross sections.
</div>
</html>

In [2]:
import Pkg; Pkg.add("Distributions"); Pkg.add("Plots")

   Resolving package versions...
   Installed LogExpFunctions ───────── v0.3.29
   Installed OrderedCollections ────── v1.7.0
   Installed StatsBase ─────────────── v0.34.4
   Installed HypergeometricFunctions ─ v0.3.25
   Installed Distributions ─────────── v0.25.116
    Updating `~/.julia/environments/v1.11/Project.toml`
  [31c24e10] + Distributions v0.25.116
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [66dad0bd] + AliasTables v1.1.3
  [34da2185] + Compat v4.16.0
  [9a962f9c] + DataAPI v1.16.0
  [864edb3b] + DataStructures v0.18.20
  [31c24e10] + Distributions v0.25.116
  [ffbed154] + DocStringExtensions v0.9.3
  [1a297f60] + FillArrays v1.13.0
  [34004b35] + HypergeometricFunctions v0.3.25
  [92d709cd] + IrrationalConstants v0.2.2
  [2ab3a3ac] + LogExpFunctions v0.3.29
  [e1d29d7a] + Missings v1.2.0
  [bac558e1] + OrderedCollections v1.7.0
  [90014a1f] + PDMats v0.11.31
  [43287f4e] + PtrArrays v1.2.1
  [1fd47b50] + QuadGK v2.11.1
  [189a3867] + Reexport v1.2.2
  [790

In [3]:
using Random, Distributions, Plots
Random.seed!(123) # Setting the seed

TaskLocalRNG()

## 2. Distribution generation 
### Example on generating multivariate Normal distribution
Use the Julia package: Distribution

In [4]:
mean = [2.,3.] # array
Cov = [0.2 0; 0 0.3] # matrix
D = MvNormal(mean, Cov) # multi-variate normal distribution
println(D)
N = 1000 # int
bunch = rand(D, N) # sampling according to distribution

FullNormal(
dim: 2
μ: [2.0, 3.0]
Σ: [0.2 0.0; 0.0 0.3]
)



2×1000 Matrix{Float64}:
 2.36148  1.50599  2.12861  1.81138  …  2.19331  2.69754  2.32349  1.99549
 2.38542  2.7716   3.12588  2.25751     3.10074  3.87942  3.26003  2.44519

Do a scatter plot

In [ ]:
gr() # grid as background
plot(bunch[1,:], bunch[2,:], seriestype = :scatter) # scatter plot, index start from 1

### K-V distribution

Under K-V (Kapchinkij-Valdimirskij) distribution, particles uniformly populate the surface of a hyper-ellipsoid in 4D phase-space
\begin{equation}
\rho(x, p_x, y, p_y) = \frac{ne}{\pi^2ab\epsilon_x\epsilon_y}\delta\left(\frac{x^2}{a^2}+\frac{y^2}{b^2}+\frac{a^2p_x^2}{\epsilon_x^2}+\frac{b^2p_y^2}{\epsilon_y^2}-1\right)
\end{equation}
- $\rho$ is the charge density
- $n$ is the number of particles per unit $z$ length (i.e. the number of particles for the volume $\pi ab \times 1$, where $\pi ab$ is the transverse beam area, $1$ is the unit $z$ length)
- $e$ is the particle's charge
- $a$ and $b$ are envelop radii of the beam. For simplicity, here we set $a=b=\epsilon_x=\epsilon_y=1$ and $x'= p_x/x, y'=p_y/y$., i.e. the ellipsoid becomes a sphere: $$ x^2+y^2+x'^2+y'^2=1$$
The state vector of a particle (in 4D phase space) is defined by
$$\zeta =
\begin{pmatrix}
x & [mm] \\ x' & [mrad] \\ y & [mm]\\ y' & [mrad]
\end{pmatrix}$$

Generating samples following a given distribtion is the [Inverse transform sampling](https://en.wikipedia.org/wiki/Inverse_transform_sampling) problem.

<html>
<div class="alert alert-warning" role="alert" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
<strong>TODO:</strong> We start from the 3D sphere case: Define a function that generates N particles on the surface of a 3D sphere, i.e. $x^2 + y^2 + z^2 = R^2$.
</div>
</html>

#### Method 1: Inverse transform with the polar coordinates $R, \theta, \phi$.

Inverse transform sampling: if $X$ is a continuous random variable with cumulative distribution function (CDF) $F_{X} = p(X \leq x)$, then the random variable $Z=F_{X}(X)$ has a uniform distribution on $[0, 1]$.
Proof:

$$ F_Z(z) = p(0 \leq Z \leq z) = p(F_X(x) \leq z) = p(x \leq F_X^{-1}(z)) = F_X(F_X^{-1}(z)) = z $$

which agrees with $Z \sim U[0,1]$.

This is the result of [Probability integral transform](https://en.wikipedia.org/wiki/Probability_integral_transform). Then we can get $X$ by $ X = F_X^{-1}(Z)$ or $F_X(X) = Z$. To write it explicitly it means

$$ F_X(x) = \int_{-\infty}^x f(\hat{x})d\hat{x} = z$$


where $f$ is the desired probability density function (PDF) of $X$. For each value $z$ sampled from $Z\sim U[0,1]$, we can calculate the corresponding $x$ from the above relation. The collection of $x$ then follow our desired distribution. 

With given $R$, we need to transform two uniformly distributed random variables $z_1$ and $z_2$ (independent) into the spherical angles $\theta$ and $\phi$ (independent). Our desired distribution is uniform on the surface, so we need to pay attention to the surface area element $dA(\theta,\phi)$.

![polar](polar.png)

$$F_{\Theta, \Phi}(\theta, \phi) = F_{\Theta}(\theta)F_{\Phi}(\phi) = ? =  z_1z_2 $$

The goal is to express $\theta$ and $\phi$ using $z_1$ and $z_2$.

In [ ]:
function gen3DSurface(N, R)
    # ...
end

<html>
<div class="alert alert-warning" role="alert" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
<strong>TODO:</strong>
   <ul>
    <li>Generate a distribution of 1000 particles.</li>
    <li>Plot the $(x,y)$ cross section.</li>
    <li>*Bonus: You can also make a 3D plot of the sphere.</li>
   </ul>
</div>
</html>

In [ ]:
N = 1000

#scatter(...)

#### Method 2: Multivariate Gaussian

<html>
<div class="alert alert-warning" role="alert" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
<strong>TODO:</strong>
   <ul>
    <li>Solve the same 3D sphere problem with the Multivariate Gaussian method showed in hint slides.
       Plot the $(x,y)$ cross section.</li>
    <li>*Bonus: The multivariate Gaussian method is actually extentable towards dimensions higher than 3. Try the 4D sphere.</li>
   </ul>
</div>
</html>

In [ ]:
function gen3DSurfaceGaussian(N,R)
    #...
end

In [ ]:
#scatter...

In [ ]:
function gen4DSurfaceGaussian(N,R)
    #...
end

### Waterbag distribution

Under waterbag distribution, particles uniformly fill the 4D hyper-ellispoid in phase-space
\begin{align*}
\rho(x,y,x',y') &= \frac{2ne}{\pi^2a^4} \\
x^2+y^2+x'^2+y'^2 &\leq a^2
\end{align*}
While $\rho$ is the charge density, the probability density of particles is thus
$$f(x,y,x',y') = \frac{2}{\pi^2 a^4}$$
with $\frac{\pi^2 a^4}{2}$ the volume of a 4D hyper sphere of radius $a$.

<html>
<div class="alert alert-warning" role="alert" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
<strong>TODO:</strong> Define a function for the 4D case using the Monte Carlo rejection method, where we generate all variables from  (−1,1)  uniform distribution, and discard the points outside the sphere.
</div>
</html>

In [ ]:
function gen4DBody(N,a)
    #...
end

<html>
<div class="alert alert-warning" role="alert" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
<strong>TODO:</strong> Generate a distribution of 1000 particles and plot the $(x,y)$ phase space.
</div>
</html>

<html>
<div class="alert alert-warning" role="alert" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
<strong>TODO:</strong>*Bonus: transformation method for 3D case
</div>
</html>

Again let's reduce to the 3D sphere case: generate N uniform 3D samples inside sphere $x^2+y^2+z^2 \leq 1$ with probability density $$f(x,y,z) = \frac{1}{Vol} = \frac{3}{4\pi r^3} = \frac{3}{4\pi}$$
    The goal is to get the same number of particles inside each unit volume:
    $$dV = r^2 \sin{\theta}drd\theta d\phi$$
![polar](polar.png)

Try to transform 3 uniformly distributed random variables $z_1, z_2, z_3$ into the above coordinates.

2. Derive from approximated Hamiltonian
$$ H_2 = \frac{1}{2}p_x^2 + \frac{1}{2}p_y^2 + \frac{1}{2}\frac{\delta^2}{\beta_0^2\gamma_0^2} $$